In [1]:
from faker import Faker
from textwrap import wrap
from itertools import chain
from functools import reduce
from collections import defaultdict

---

In [3]:
fake = Faker()

In [4]:
word = fake.word()

---

In [6]:
def flatten(listOfLists: list[list]) -> list:
    return list(chain.from_iterable(listOfLists))

In [7]:
def calculatePercentageOfNumber(percent: int, whole: int) -> int:
    return round((percent / 100) * whole)

---

## Divide word into syllables for search

### Spin word

#### One spin

In [12]:
def spinWord(word: str) -> str:
    firstLetter = word[0]
    rest = word[1:]
    return ''.join([rest, firstLetter])

#### Spin around

In [14]:
def spinWordAround(word: str) -> list[str]:
    originalWord = word
    spunList = [originalWord]
    spun = spinWord(word)

    while originalWord != spun:
        spunList.append(spun)
        spun = spinWord(spun)

    return spunList

### Chunk

#### syllables

In [17]:
def getSyllables(word: str, syllableLength: int) -> list[str]:
    syllables = wrap(word, syllableLength)
    return list(filter(lambda syllable: len(syllable) == syllableLength, syllables))

In [18]:
def getPossibleSyllables(wordSpins, syllableLength):
    for word in wordSpins:
        yield from sorted(getSyllables(word, syllableLength))

In [19]:
def filterSyllablesByWord(syllables: list[str], word: str) -> list[str]:
    existed = list(filter(lambda syllable: syllable in word, syllables))
    unique = reduce(lambda acc, i: acc + [i] if i not in acc else acc, existed, [])
    return unique

### Divide word by Spining and Chunking

In [21]:
def divideIntoSyllables(word: str, syllableLength) -> list[str]:
    wordSpins = spinWordAround(word)
    syllables = getPossibleSyllables(wordSpins, syllableLength)
    unique = filterSyllablesByWord(syllables, word)

    return unique

### Frequencies

#### Count syllables

In [24]:
def countSyllables(words: list[str], syllableLength: int) -> dict[str, int]:
    syllablesFrequencies = defaultdict(int)
    
    for word in words:
        for syllable in divideIntoSyllables(word, syllableLength):
                syllablesFrequencies[syllable] += 1

    return syllablesFrequencies

#### Merge and sort

In [26]:
def getSyllablesFrequencies(words: list[str], syllableLength: int = 2) -> dict[str, int]:
    syllablesFrequencies = countSyllables(words, syllableLength)
    sortedAlphabetically = sorted(syllablesFrequencies)
    sortedByFrequency = sorted(sortedAlphabetically, key=syllablesFrequencies.get, reverse=True)
    syllablesFrequencies = [(syllable, syllablesFrequencies[syllable]) for syllable in sortedByFrequency]
    
    return dict(syllablesFrequencies)

#### Filter by percentage

In [45]:
def filterSyllablesFrequenciesByPercentageFrequency(wordsWithFrequencies: dict[str, int], minFrequencyPercentage: int = 0) -> dict[str, int]:
    maxValue = max(wordsWithFrequencies.values())
    minFrequency = calculatePercentageOfNumber(minFrequencyPercentage, maxValue)
    filteredWordsWithFrequencies = filter(lambda wordFrequency: wordFrequency[1] > minFrequency, wordsWithFrequencies.items())
    
    return dict(filteredWordsWithFrequencies)